In [1]:
!pip install yfinance

In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [ ]:
%%writefile web_app_v3.py
# Import required libraries
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# Function calling local CSS sheet
def local_css(file_name):
    with open(file_name) as f:
        st.sidebar.markdown(f'<style>{f.read()}</style>', unsafe_allow_html=True)

# Local CSS sheet
local_css("style.css")

# Ticker search feature in sidebar
st.sidebar.subheader("""Stock Search Web App""")
selected_stock = st.sidebar.text_input("Enter a valid stock ticker...", "GOOG")

# Main function
def main():
    st.title("Stock Price Analysis and Prediction")
    st.subheader("""Daily **closing price** for """ + selected_stock)
    # Get data on the searched ticker
    stock_data = yf.Ticker(selected_stock)
    # Get historical data for the searched ticker
    stock_df = stock_data.history(period='1d', start='2020-01-01', end=None)
    # Print line chart with daily closing prices for the searched ticker
    st.line_chart(stock_df.Close)

    st.subheader("""Last **closing price** for """ + selected_stock)
    # Define variable today
    today = datetime.today().strftime('%Y-%m-%d')
    # Get current date data for the searched ticker
    stock_lastprice = stock_data.history(period='1d', start=today, end=today)
    # Get current date closing price for the searched ticker
    last_price = stock_lastprice.Close.values[0]
    # If the market is closed on the current date, print that there is no data available
    if pd.isnull(last_price):
        st.write("No data available at the moment")
    else:
        st.write(last_price)

    # Get daily volume for the searched ticker
    st.subheader("""Daily **volume** for """ + selected_stock)
    st.line_chart(stock_df.Volume)

    # Additional information feature in sidebar
    st.sidebar.subheader("""Display Additional Information""")
    # Checkbox to display stock actions for the searched ticker
    actions = st.sidebar.checkbox("Stock Actions")
    if actions:
        st.subheader("""Stock **actions** for """ + selected_stock)
        display_action = (stock_data.actions)
        if display_action.empty == True:
            st.write("No data available at the moment")
        else:
            st.write(display_action)

    # Checkbox to display quarterly financials for the searched ticker
    financials = st.sidebar.checkbox("Quarterly Financials")
    if financials:
        st.subheader("""**Quarterly financials** for """ + selected_stock)
        display_financials = (stock_data.quarterly_financials)
        if display_financials.empty == True:
            st.write("No data available at the moment")
        else:
            st.write(display_financials)

    # Checkbox to display list of institutional shareholders for searched ticker
    major_shareholders = st.sidebar.checkbox("Institutional Shareholders")
    if major_shareholders:
        st.subheader("""**Institutional investors** for """ + selected_stock)
        display_shareholders = (stock_data.institutional_holders)
        if display_shareholders.empty == True:
            st.write("No data available at the moment")
        else:
            st.write(display_shareholders)

    # Checkbox to display quarterly balance sheet for searched ticker
    balance_sheet = st.sidebar.checkbox("Quarterly Balance Sheet")
    if balance_sheet:
        st.subheader("""**Quarterly balance sheet** for """ + selected_stock)
        display_balancesheet = (stock_data.quarterly_balance_sheet)
        if display_balancesheet.empty == True:
            st.write("No data available at the moment")
        else:
            st.write(display_balancesheet)

    # Checkbox to display quarterly cashflow for searched ticker
    cashflow = st.sidebar.checkbox("Quarterly Cashflow")
    if cashflow:
        st.subheader("""**Quarterly cashflow** for """ + selected_stock)
        display_cashflow = (stock_data.quarterly_cashflow)
        if display_cashflow.empty == True:
            st.write("No data available at the moment")
        else:
            st.write(display_cashflow)

    # Create input fields for entering stock ticker symbols
    st.sidebar.header("Enter Ticker Symbols")
    ticker1 = st.sidebar.text_input("Ticker 1", value="AAPL")
    ticker2 = st.sidebar.text_input("Ticker 2", value="MSFT")
    ticker3 = st.sidebar.text_input("Ticker 3", value="GOOGL")
    ticker4 = st.sidebar.text_input("Ticker 4", value="AMZN")

    # Fetch historical data using yfinance
    def get_data(ticker):
        stock_data = yf.download(ticker, period="1y", progress=False)
        return stock_data

    # Get data for the selected ticker symbols
    data1 = get_data(ticker1)
    data2 = get_data(ticker2)
    data3 = get_data(ticker3)
    data4 = get_data(ticker4)

    # Plot the historical closing prices
    st.subheader("Stocks Closing Price Comparison")

    # Create a DataFrame containing closing prices of all selected stocks
    closing_prices = pd.concat([data1['Close'], data2['Close'], data3['Close'], data4['Close']], axis=1)
    closing_prices.columns = [ticker1, ticker2, ticker3, ticker4]

    # Plot the closing prices
    plt.figure(figsize=(12, 6))
    for column in closing_prices.columns:
        plt.plot(closing_prices.index, closing_prices[column], label=column)

    plt.title("Closing Price Comparison")
    plt.xlabel("Date")
    plt.ylabel("Closing Price")
    plt.legend(loc="upper left")
    st.pyplot(plt)

    # Show raw data
    st.subheader("Raw Data")
    st.write("Closing Prices")
    st.write(closing_prices)

    # Stock Price Prediction using LSTM
    st.title("Stock Price Prediction using LSTM")

    # Sidebar
    st.sidebar.title("Settings")
    ticker_lstm = st.sidebar.text_input("Enter Stock Ticker Symbol (e.g., AAPL)", "AAPL")
    start_date_lstm = st.sidebar.date_input("Start Date", datetime(2020, 1, 1))
    end_date_lstm = st.sidebar.date_input("End Date (Today)", datetime.today())
    forecast_days_lstm = st.sidebar.number_input("Number of Forecast Days", min_value=1, max_value=365, value=30)
    show_data_lstm = st.sidebar.checkbox("Show Historical Data")

    # Download historical data for LSTM
    data_lstm = yf.download(ticker_lstm, start=start_date_lstm, end=end_date_lstm)

    if show_data_lstm:
        st.subheader("Historical Data for LSTM")
        st.write(data_lstm)

    # Data preprocessing for LSTM
    data_lstm['Date'] = data_lstm.index
    data_lstm['Close'] = data_lstm['Close'].astype(float)

    # Create a dataset for training and testing for LSTM
    data_train_lstm = data_lstm[['Date', 'Close']].copy()
    data_train_lstm.set_index('Date', inplace=True)

    # Normalize the data
    scaler_lstm = MinMaxScaler()
    data_train_scaled_lstm = scaler_lstm.fit_transform(data_train_lstm)

    # Create sequences for training for LSTM
    sequence_length_lstm = 20  # You can adjust this value
    X_lstm, y_lstm = [], []

    for i in range(sequence_length_lstm, len(data_train_scaled_lstm)):
        X_lstm.append(data_train_scaled_lstm[i-sequence_length_lstm:i, 0])
        y_lstm.append(data_train_scaled_lstm[i, 0])

    X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

    # Split your data into training and validation sets
    split_ratio = 0.8
    split_index = int(len(X_lstm) * split_ratio)

    X_train_lstm, y_train_lstm = X_lstm[:split_index], y_lstm[:split_index]
    X_val_lstm, y_val_lstm = X_lstm[split_index:], y_lstm[split_index:]

    # Build the LSTM model
    model_lstm = Sequential()
    model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)))
    model_lstm.add(LSTM(units=50))
    model_lstm.add(Dense(units=1))
    model_lstm.compile(optimizer='adam', loss='mean_squared_error')
    model_lstm.fit(X_train_lstm, y_train_lstm, epochs=5, batch_size=32)

    # Train the model using X_val_lstm and y_val_lstm for validation
    history_lstm = model_lstm.fit(X_train_lstm, y_train_lstm, epochs=5, batch_size=32, validation_data=(X_val_lstm, y_val_lstm))

    # Evaluate the model on the validation data
    eval_loss = model_lstm.evaluate(X_val_lstm, y_val_lstm)


    # Display loss and accuracy on validation data
    st.subheader("Loss and Accuracy on Validation Data")
    st.write(f"Loss on Validation Data: {eval_loss:.4f}")
    last_date_lstm = data_train_lstm.index[-1]
    future_dates_lstm = [last_date_lstm + timedelta(days=i) for i in range(1, forecast_days_lstm + 1)]
    future_data_lstm = data_train_lstm.iloc[-sequence_length_lstm:, :].values
    future_predictions_lstm = []

    # Predict future prices for LSTM
    for i in range(forecast_days_lstm):
        prediction_lstm = model_lstm.predict(future_data_lstm.reshape(1, -1, 1))
        future_data_lstm = np.append(future_data_lstm, prediction_lstm)
        future_data_lstm = future_data_lstm[1:]
        future_predictions_lstm.append(prediction_lstm[0][0])

    # Inverse transform the predictions to the original scale
    future_predictions_lstm = scaler_lstm.inverse_transform(np.array(future_predictions_lstm).reshape(-1, 1))

    # Create a DataFrame for future predictions for LSTM
    future_data_lstm = pd.DataFrame({'Date': future_dates_lstm, 'Predicted Price': future_predictions_lstm.flatten()})
    future_data_lstm.set_index('Date', inplace=True)

    # Calculate accuracy for LSTM
    st.subheader("Accuracy for LSTM")

    # Create a DataFrame containing actual and predicted prices for LSTM
    actual_prices_lstm = data_lstm['Close'][-forecast_days_lstm:].values
    predicted_prices_lstm = future_data_lstm['Predicted Price'].values

    # Calculate accuracy as a percentage
    accuracy_lstm = (1 - (np.abs(actual_prices_lstm - predicted_prices_lstm) / actual_prices_lstm)) * 100
    average_accuracy_lstm = np.mean(accuracy_lstm)

    # Display average accuracy
    st.write(f"Average Accuracy on Predictions: {average_accuracy_lstm:.2f}%")



    # Predict future stock prices for LSTM
    st.subheader("Stock Price Prediction for LSTM")

    # Prepare data for prediction for LSTM
    last_date_lstm = data_train_lstm.index[-1]
    future_dates_lstm = [last_date_lstm + timedelta(days=i) for i in range(1, forecast_days_lstm + 1)]
    future_data_lstm = data_train_lstm.iloc[-sequence_length_lstm:, :].values
    future_predictions_lstm = []

    # Predict future prices for LSTM
    for i in range(forecast_days_lstm):
        prediction_lstm = model_lstm.predict(future_data_lstm.reshape(1, -1, 1))
        future_data_lstm = np.append(future_data_lstm, prediction_lstm)
        future_data_lstm = future_data_lstm[1:]
        future_predictions_lstm.append(prediction_lstm[0][0])

    # Inverse transform the predictions to the original scale
    future_predictions_lstm = scaler_lstm.inverse_transform(np.array(future_predictions_lstm).reshape(-1, 1))

    # Create a DataFrame for future predictions for LSTM
    future_data_lstm = pd.DataFrame({'Date': future_dates_lstm, 'Predicted Price': future_predictions_lstm.flatten()})
    future_data_lstm.set_index('Date', inplace=True)

    # Display the future price predictions for LSTM
    st.line_chart(future_data_lstm)

if __name__ == "__main__":
    main()



Writing web_app_v3.py


In [ ]:
!streamlit run web_app_v3.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.23.43.97:8501

